In [1]:
# Imports 
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


from scipy import stats

# The Problem

So the problem is 'how do I know if wine is good?'  
Specifically, when in a shop looking for something to buy. 

There are a few ways I could determine this. 

# The Solution(s)?

1. I could do what I did with the other wine related datasets and train a tree model to get which are the ~3-5 most important factors, and what ranges they should lie in. While I think this will work - and I intend to do it - it also tends to always end up focussing on only one particular ordered set of questions with the same answers; country A, variety B, price C, etc. which doesn't tell me anything about good wines from country F (and I assume that country F must have some good wine or other? 

2. I could use some sort of relatively simple model (e.g. linear regression) and try to determine from the coefficients what features are important, and what values are better, but attempts to determine feature relevance in that way usually fail, as coefficients do not correlate with feature importance. I could use some sort of feature selection to further reduce the number of features I have, and make the result easier to understand somehow? The effectiveness of that is unclear, but unlikely. Then again, [scikit-learn's `SelectFromModel`](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SelectFromModel.html "scikit-learn docs") exists to do just that - automatically - so maybe that's worth a look. This still doesn't actually tell me what _values_ of features mean good wine though, so I probably won't do this. 

3. I could train an accurate, black box model that classifies wine, then put that model into a phone app, and use it to give me a likely score (or range of scores) when I manually enter the features of the wine I am looking at on the shelf. The more data I enter, the more accurate the score. This seems like an approach that will provide an actual, physical solution to my problem, but will require a lot more work. 

# The Plan

The first thing I'm going to do is the easiest - solution 1: decision tree models.  
Of course the _zeroth_ thing I'm going to do is load and preprocess the data. 

In [2]:
# Inital data loading 
data = pd.read_csv('../input/wine-reviews/winemag-data-130k-v2.csv', index_col=0)
target_data = data['points']
feature_data = data.drop('points', axis=1)

# Creation of new feature 'vintage'
titles = feature_data.title.copy(deep=True)
years = titles.replace(r'.*((19|20)[0-9]{2}).*', r'\1', regex=True).replace(r'[A-Za-z]|[\D]|\s|(?:(?<!\d)\d{1}(?!\d))|(?:(?<!\d)\d{2}(?!\d))|(?:(?<!\d)\d{3}(?!\d))', '', regex=True)
filtered_years = years.replace(r'(1503|1607|1821|1827|1847|1868|1872|1868|1872|1882|1887|2067)', '', regex=True)
vintage = filtered_years

# Removal of unusable features (I can't find the reviewers description 
# on the bottle, can I?) and addition of new feature 'vintage'
raw_reduced_features = feature_data.drop(labels=['description', 'designation', 'title'], axis=1)
raw_reduced_features['vintage'] = vintage.copy(deep=False).replace('', '0').astype(int)

In [3]:
# Separate out all the features so I can transform them individually, 
# and then put them back together into different datasets
country = raw_reduced_features.country
price = raw_reduced_features.price
province = raw_reduced_features.province
region_1 = raw_reduced_features.region_1
region_2 = raw_reduced_features.region_2
taster_name = raw_reduced_features.taster_name
taster_twitter_handle = raw_reduced_features.taster_twitter_handle
variety = raw_reduced_features.variety
winery = raw_reduced_features.winery
vintage = raw_reduced_features.vintage

In [4]:
reduced_features_non_null_data = raw_reduced_features.dropna(subset=['country', 'province', 'region_1', 'taster_name', 'variety', 'winery'], axis='index', how='any').copy(deep=False)
print('Number of rows left with no null values after dropping region_2 and taster_twitter_handle: {}'.format(reduced_features_non_null_data.shape[0]))

Number of rows left with no null values after dropping region_2 and taster_twitter_handle: 82847


In [5]:
# Separate out all the features so I can transform them individually, 
# and then put them back together into different datasets
country_non_null = reduced_features_non_null_data.country
price_non_null = reduced_features_non_null_data.price
province_non_null = reduced_features_non_null_data.province
region_1_non_null = reduced_features_non_null_data.region_1
region_2_non_null = reduced_features_non_null_data.region_2
taster_name_non_null = reduced_features_non_null_data.taster_name
taster_twitter_handle_non_null = reduced_features_non_null_data.taster_twitter_handle
variety_non_null = reduced_features_non_null_data.variety
winery_non_null = reduced_features_non_null_data.winery
vintage_non_null = reduced_features_non_null_data.vintage

## Ordinal Numerical Encoding

In [6]:
# country
# nulls
label_encoder_country = LabelEncoder()
label_encoder_country.fit(country.astype(str))
encoded_countries = label_encoder_country.transform(country.astype(str))

# non-nulls
label_encoder_non_null_country = LabelEncoder()
label_encoder_non_null_country.fit(country_non_null.astype(str))
encoded_non_null_countries = label_encoder_non_null_country.transform(country_non_null.astype(str))

In [7]:
# province
# nulls
label_encoder_province = LabelEncoder()
label_encoder_province.fit(province.astype(str))
encoded_provinces = label_encoder_province.transform(province.astype(str))

# non-nulls
label_encoder_non_null_province = LabelEncoder()
label_encoder_non_null_province.fit(province_non_null.astype(str))
encoded_non_null_provinces = label_encoder_non_null_province.transform(province_non_null.astype(str))

In [8]:
# region_1
# nulls
label_encoder_region_1 = LabelEncoder()
label_encoder_region_1.fit(region_1.astype(str))
encoded_region_1s = label_encoder_region_1.transform(region_1.astype(str))

# non-nulls
label_encoder_non_null_region_1 = LabelEncoder()
label_encoder_non_null_region_1.fit(region_1_non_null.astype(str))
encoded_non_null_region_1s = label_encoder_non_null_region_1.transform(region_1_non_null.astype(str))

In [9]:
# region_2
# nulls
label_encoder_region_2 = LabelEncoder()
label_encoder_region_2.fit(region_2.astype(str))
encoded_region_2s = label_encoder_region_2.transform(region_2.astype(str))

# non-nulls
label_encoder_non_null_region_2 = LabelEncoder()
label_encoder_non_null_region_2.fit(region_2_non_null.astype(str))
encoded_non_null_region_2s = label_encoder_non_null_region_2.transform(region_2_non_null.astype(str))

In [10]:
# taster_name
# nulls
label_encoder_taster_name = LabelEncoder()
label_encoder_taster_name.fit(taster_name.astype(str))
encoded_taster_names = label_encoder_taster_name.transform(taster_name.astype(str))

# non-nulls
label_encoder_non_null_taster_name = LabelEncoder()
label_encoder_non_null_taster_name.fit(taster_name_non_null.astype(str))
encoded_non_null_taster_names = label_encoder_non_null_taster_name.transform(taster_name_non_null.astype(str))

In [11]:
# taster_twitter_handle
# nulls
label_encoder_taster_twitter_handle = LabelEncoder()
label_encoder_taster_twitter_handle.fit(taster_twitter_handle.astype(str))
encoded_taster_twitter_handles = label_encoder_taster_twitter_handle.transform(taster_twitter_handle.astype(str))

# non-nulls
label_encoder_non_null_taster_twitter_handle = LabelEncoder()
label_encoder_non_null_taster_twitter_handle.fit(taster_twitter_handle_non_null.astype(str))
encoded_non_null_taster_twitter_handles = label_encoder_non_null_taster_twitter_handle.transform(taster_twitter_handle_non_null.astype(str))

In [12]:
# variety
# nulls
label_encoder_variety = LabelEncoder()
label_encoder_variety.fit(variety.astype(str))
encoded_varieties = label_encoder_variety.transform(variety.astype(str))

# non-nulls
label_encoder_non_null_variety = LabelEncoder()
label_encoder_non_null_variety.fit(variety_non_null.astype(str))
encoded_non_null_varieties = label_encoder_non_null_variety.transform(variety_non_null.astype(str))

In [13]:
# winery
# nulls
label_encoder_winery = LabelEncoder()
label_encoder_winery.fit(winery.astype(str))
encoded_wineries = label_encoder_winery.transform(winery.astype(str))

# non-nulls
label_encoder_non_null_winery = LabelEncoder()
label_encoder_non_null_winery.fit(winery_non_null.astype(str))
encoded_non_null_wineries = label_encoder_non_null_winery.transform(winery_non_null.astype(str))

## One Hot Encoding

In [14]:
# country
# nulls
one_hot_country = OneHotEncoder(sparse=False)
one_hot_country.fit(country.astype(str).values.reshape(-1,1))
one_hot_countries = one_hot_country.transform(country.astype(str).values.reshape(-1,1))

In [15]:
# region_2
one_hot_region_2 = OneHotEncoder(sparse=False)
one_hot_region_2.fit(region_2.astype(str).values.reshape(-1,1))
one_hot_region_2s = one_hot_region_2.transform(region_2.astype(str).values.reshape(-1,1))

In [16]:
# taster_name
one_hot_taster_name = OneHotEncoder(sparse=False)
one_hot_taster_name.fit(taster_name.astype(str).values.reshape(-1,1))
one_hot_taster_names = one_hot_taster_name.transform(taster_name.astype(str).values.reshape(-1,1))

In [17]:
# taster_twitter_handle
one_hot_taster_twitter_handle = OneHotEncoder(sparse=False)
one_hot_taster_twitter_handle.fit(taster_twitter_handle.astype(str).values.reshape(-1,1))
one_hot_taster_twitter_handles = one_hot_taster_twitter_handle.transform(taster_twitter_handle.astype(str).values.reshape(-1,1))

## Creating Datasets

In [18]:
# Label encoded feature data that contains nulls
feature_data_label_encoded = raw_reduced_features.copy(deep=False)
feature_data_label_encoded['vintage'] = vintage

feature_data_label_encoded.country = encoded_countries
feature_data_label_encoded.province = encoded_provinces
feature_data_label_encoded.region_1 = encoded_region_1s
feature_data_label_encoded.region_2 = encoded_region_2s
feature_data_label_encoded.taster_name = encoded_taster_names
feature_data_label_encoded.taster_twitter_handle = encoded_taster_twitter_handles
feature_data_label_encoded.variety = encoded_varieties
feature_data_label_encoded.winery = encoded_wineries

In [19]:
# Label encoded feature data that does not contain nulls
feature_data_label_encoded_non_null = reduced_features_non_null_data.copy(deep=False)
feature_data_label_encoded_non_null['vintage'] = vintage

feature_data_label_encoded_non_null.country = encoded_non_null_countries
feature_data_label_encoded_non_null.province = encoded_non_null_provinces
feature_data_label_encoded_non_null.region_1 = encoded_non_null_region_1s
feature_data_label_encoded_non_null.region_2 = encoded_non_null_region_2s
feature_data_label_encoded_non_null.taster_name = encoded_non_null_taster_names
feature_data_label_encoded_non_null.taster_twitter_handle = encoded_non_null_taster_twitter_handles
feature_data_label_encoded_non_null.variety = encoded_non_null_varieties
feature_data_label_encoded_non_null.winery = encoded_non_null_wineries

In [20]:
# Mixed encoding feature data
feature_data_mixed_encoding = raw_reduced_features.copy(deep=False)
feature_data_mixed_encoding = feature_data_mixed_encoding.drop(labels=['country', 'region_2', 'taster_name', 'taster_twitter_handle'], axis=1)
feature_data_mixed_encoding['vintage'] = vintage

# Label encoding
feature_data_mixed_encoding.province = encoded_provinces
feature_data_mixed_encoding.region_1 = encoded_region_1s
feature_data_mixed_encoding.variety = encoded_varieties
feature_data_mixed_encoding.winery = encoded_wineries

# One hot encoding
one_hot_countries_df = pd.DataFrame(data=one_hot_countries, columns=one_hot_country.categories_[0])
one_hot_countries_df = one_hot_countries_df.rename(columns={"nan":"country_nan"})
feature_data_mixed_encoding = pd.concat([feature_data_mixed_encoding, one_hot_countries_df], axis=1)

one_hot_region_2s_df = pd.DataFrame(data=one_hot_region_2s, columns=one_hot_region_2.categories_[0])
one_hot_region_2s_df = one_hot_region_2s_df.rename(columns={"nan":"region_2_nan"})
feature_data_mixed_encoding = pd.concat([feature_data_mixed_encoding, one_hot_region_2s_df], axis=1)

one_hot_taster_names_df = pd.DataFrame(data=one_hot_taster_names, columns=one_hot_taster_name.categories_[0])
one_hot_taster_names_df = one_hot_taster_names_df.rename(columns={"nan":"taster_name_nan"})
feature_data_mixed_encoding = pd.concat([feature_data_mixed_encoding, one_hot_taster_names_df], axis=1)

one_hot_taster_twitter_handles_df = pd.DataFrame(data=one_hot_taster_twitter_handles, columns=one_hot_taster_twitter_handle.categories_[0])
one_hot_taster_twitter_handles_df = one_hot_taster_twitter_handles_df.rename(columns={"nan":"taster_twitter_handle_nan"})
feature_data_mixed_encoding = pd.concat([feature_data_mixed_encoding, one_hot_taster_twitter_handles_df], axis=1)

### Test/Train Splitting

In [21]:
# Label Encoded full data test train split
X_train_label_encoded, X_test_label_encoded, y_train_label_encoded, y_test_label_encoded = train_test_split(feature_data_label_encoded, target_data, random_state=4)

In [22]:
# Label Encoded non-null data test train split
target_data_non_null = target_data[feature_data_label_encoded_non_null.index]
X_train_label_encoded_non_null, X_test_label_encoded_non_null, y_train_label_encoded_non_null, y_test_label_encoded_non_null = train_test_split(feature_data_label_encoded_non_null, target_data_non_null, random_state=4)

In [23]:
# Mixed encoding full data test train split
X_train_mixed_encoding, X_test_mixed_encoding, y_train_mixed_encoding, y_test_mixed_encoding = train_test_split(feature_data_mixed_encoding, target_data, random_state=4)

### Standardisation of Originally Numerical Data
The originally numerical data being the price, and the vintage, both of which have massive skew that is likely to interfere with any model training. 

In [24]:
# feature_data_label_encoded.iloc[X_train_label_encoded.index.values,:]
X_train_label_encoded

,country,price,province,region_1,region_2,taster_name,taster_twitter_handle,variety,winery,vintage
88977,31,13.0,108,1229,17,15,10,450,15652,2015
54172,40,120.0,51,1009,12,18,9,62,1825,2014
18007,22,44.0,286,79,17,19,15,383,8222,2007
42295,0,20.0,216,589,17,12,12,473,2406,2014
28868,22,14.0,68,662,17,9,4,347,2323,2015
67958,22,35.0,374,209,17,9,4,506,1614,2011
13027,40,47.0,268,1217,16,14,7,440,9826,2006
59668,40,20.0,51,155,11,7,3,7,13844,2013
33273,31,19.0,108,1229,17,15,10,450,5050,2013
67420,15,NaN,36,817,17,15,10,62,3082,2009


In [25]:
# Label Encoded full data standardisation
label_encoded_feature_data_to_be_standardised_X_train = X_train_label_encoded[['price', 'vintage']]
label_encoded_feature_data_to_be_standardised_X_test = X_test_label_encoded[['price', 'vintage']]
label_encoded_standard_scaler = StandardScaler()
label_encoded_standard_scaler.fit(label_encoded_feature_data_to_be_standardised_X_train)

X_train_label_encoded_standardised = label_encoded_standard_scaler.transform(label_encoded_feature_data_to_be_standardised_X_train)
X_test_label_encoded_standardised = label_encoded_standard_scaler.transform(label_encoded_feature_data_to_be_standardised_X_test)

X_train_label_encoded_standardised_df = pd.DataFrame(data=X_train_label_encoded_standardised, columns=['price', 'vintage'])
X_test_label_encoded_standardised_df = pd.DataFrame(data=X_test_label_encoded_standardised, columns=['price', 'vintage'])

X_train_label_encoded = X_train_label_encoded.drop(['price', 'vintage'], axis=1)
X_train_label_encoded['price'] = X_train_label_encoded_standardised_df['price']
X_train_label_encoded['vintage'] = X_train_label_encoded_standardised_df['vintage']

# X_train_label_encoded.iloc[13896,:]
# price[13896]
# vintage[13896]
# label_encoded_feature_data_to_be_standardised_X_train.loc[13896,:]
# feature_data.loc[13896,:]
from scipy import stats
X_train_label_encoded_standardised_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97478 entries, 0 to 97477
Data columns (total 2 columns):
price      90701 non-null float64
vintage    97478 non-null float64
dtypes: float64(2)
memory usage: 1.5 MB


In [26]:
# Label Encoded non-null data standardisation
label_encoded_non_null_standard_scaler = StandardScaler()
label_encoded_non_null_standard_scaler.fit(X_train_label_encoded_non_null)

X_train_label_necoded_non_null_standardised = label_encoded_non_null_standard_scaler.transform(X_train_label_encoded_non_null)
X_test_label_necoded_non_null_standardised = label_encoded_non_null_standard_scaler.transform(X_test_label_encoded_non_null)

In [27]:
# Mixed encoding full data standardisation

XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

1. ~~numerisation of textual data~~
2. ~~test/train splitting~~
3. standardisation of originally numerical data

In [28]:
# Numerical feature standardisation


In [29]:
# Make nullless dataset
non_null_reduced_feature_data = raw_reduced_features.dropna(subset=['country', 'price', 'province', 'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'variety', 'winery', 'vintage'], axis='index', how='any')